<a href="https://colab.research.google.com/github/hsieh2000/HW/blob/main/exp_annotaton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#匯入套件
import time
import json 
import math
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime
import os
import re
from fake_useragent import UserAgent
import pymysql
import random
import warnings 
import numpy as np                                             

config_path = "../../setting.json"                                      #設定setting.json路徑
with open(config_path) as f:                                            
    config = json.load(f)                                               #以json形式讀取setting.json
    root = config["root"]                                               #取得root路徑

# 建立資料夾
newest_path = f'{root}最新檔/'                                          #設定最新檔匯出路徑
log_path = f'{root}近五年費用率與報酬率記錄檔/'                             #設定記錄檔輸出路徑
dir = ["csv/", "txt/", "err/"]                                          #設定資料夾名稱

for i in dir:
  if not os.path.isdir(log_path+i):                                     #判斷是否資料夾存在，如果不存在則建立
    os.makedirs(log_path+i)

if not os.path.isdir(newest_path):                                      #判斷是否資料夾存在，如果不存在則建立
    os.mkdir(newest_path)

# 路徑設定
csv_path = log_path+dir[0]
txt_path = log_path+dir[1]
err_path = log_path+dir[2]
basicInfo_path = f"{newest_path}境外基金基本資訊.csv"

# 參數設定
delay = random.uniform(1, 5)                                            #設定隨機數1至5
warnings.filterwarnings("ignore")                                       #忽略警告訊息
useragent = UserAgent()                                                 #調用 UserAgent() 使用者代理
now = datetime.datetime.now().strftime("%Y%m%d")                        #取得執行當天日期
filename = f'近五年費用率與報酬率'                                         #設定檔名

c = pd.read_csv(f'{basicInfo_path}')                                    #讀取最新基本資訊檔

header = {
    'Host' : 'announce.fundclear.com.tw',
    'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    "Referer": "https://announce.fundclear.com.tw",   
    "user-agent" : useragent.random
}                                                                       #設定requests header

class expNre(object):
    def __init__(self):
        a = self.crawler()
        a = self.store(a)
        self.to_mysql(a)

    def get_source(self, url):                                          #當爬蟲失敗時會自動嘗試重新爬取，最多三次
        times = 0
        while times<3 :
            try:
                res=requests.get(url,headers = header, timeout=20)      #取得網頁內容，timeout設為20秒
                soup = BeautifulSoup(res.text, 'html.parser')           #以html.parser解析網站
                soup.select("td.TableHead")[1]#確認有值
                return soup
            except:
                times+=1
                print(f"try {times} time, {url}")
                pass


    def crawler(self):
        J = []
        #可以加上報錯
        with open(f'{err_path}.txt', mode = "w") as e:
            e.write("錯誤連結:\n")
        with open(f'{txt_path}{filename}{now}.txt', mode = "w") as f:
            f.write("{data:[")
        for j,i in enumerate(c['基金代碼']):
            url = f"https://announce.fundclear.com.tw/MOPSFundWeb/A01_12.jsp?fundId={i}"

            soup = self.get_source(url)
            #可以加上try,except
            try:
                TableHead = list(map(lambda x: x.text, soup.select("td.TableHead")))            #篩選欄位名稱
                rate = list(map(lambda x: x.text, soup.select('td[align="right"]')))            #篩選數值
                year = list(map(lambda x: x.text, soup.select('td[width="17%"]')))              #篩選年份
                exp = rate[:5]                                                                  #區分出費用率
                re = rate[5:]                                                                   #區分出報酬率
                Json = list(map(lambda x,y,z:{
                    '基金代碼':i,
                    TableHead[0]:x,
                    TableHead[1]:y,
                    TableHead[2]:z
                }, year,exp,re))                                                                #將資料轉為list of json 形式
                for x in Json:                                                                  #將資料寫入txt檔
                    with open(f'{txt_path}{filename}{now}.txt', mode = "a") as f:
                        f.write(str(x))
                        f.write(',')

                J = J+Json
            except:                                                                             #假如資訊獲取失敗則將錯誤連結寫入err資料夾
                print(f"{url} 出錯")
                with open(f'{err_path}.txt', mode = "a") as e:
                    e.write(f"{url}\n")
            print(f"{j}/{len(c)}")
            time.sleep(delay)
            #測試
            # if j == 10:
            #   break  

        with open(f'{txt_path}{filename}{now}.txt', mode = "a") as f:
            f.write("]}")
        return J

    def store(self, J):
        df = pd.DataFrame(data = J)                                                             #將資料轉為Dataframe形式
        df.to_csv(f"{csv_path}{filename}{now}.csv", encoding ="utf-8")                          #匯出資料
        df.to_csv(f"{newest_path}{filename}.csv", encoding ="utf-8")

        return df

    def to_mysql(self, df):                                                                     #資料庫連線
        db_settings = {
            "host": "163.14.137.58",
            "port": 7733,
            "user": "ftca",
            "password": "ftca",
            "db": "raw_data",
            "charset": "utf8"
        }
        # 建立Cursor物件
        conn = pymysql.connect(**db_settings)

        with conn.cursor() as cursor:
            # 新增資料SQL語法
            command = """INSERT INTO expence_return(FUND_ID, `YEAR`, EXPENCE, `RETURN`)VALUES(%s, %s, %s, %s)"""
            for i in range(len(df)):                                                                                                #將資料匯入資料庫
                cursor.execute(
                    command, (df.iloc[i][0], df.iloc[i][1], df.iloc[i][2], df.iloc[i][3]))
                print(f"第{i}筆成功")
            # 儲存變更
            print("saved data to mysql!")
            conn.commit()


if __name__ == "__main__":
    E = expNre()